In [1]:
import os 
import matplotlib.pyplot as plot
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import pandas as pd
import numpy as np 

In [2]:
def get_data(file_name):
    with open(file_name, 'r') as array3D:
        content = array3D.read()
    
    # Sort data into two variables , one with the image data (scan_array)
    # and another with the meta data for the image. 
    scan_array = []
    meta_data = []

    lines = content.splitlines()#[6:]
    for line in lines:
        columns = line.split('\t')
        if len(columns) < 20:
            meta_data.append(columns)
            continue 
        scan_array.append(columns[0:len(columns)-1]) 

    # convert the scan_array list into a float array. 
    image_array = np.array(scan_array, dtype = np.float)
    
    max_fluorescence = np.max(image_array)
    
    return(image_array, meta_data, max_fluorescence)

In [3]:
def xyz_scales(meta_data):
    # Extract the XYZ scaling information from the meta_data list. 
    xyz = meta_data[10]
    XYZscaling = np.array(xyz[0:len(xyz)-1], dtype = np.float)
    #find the maximum scan length
    scan_maxima = [XYZscaling[2]-XYZscaling[1], 
                   XYZscaling[5]- XYZscaling[4],
                   XYZscaling[8]- XYZscaling[7]]
    #find the size of each voxel/pixel
    voxel_size = np.zeros([3])
    for i in range(len(scan_maxima)):
        if scan_maxima[i] == 0:
            continue 
        else:
            voxel_size[i] = scan_maxima[i]/int(XYZscaling[i*3+3]) 

    # Create the pixle arrays for both the x and y range.
    x_scan = np.linspace(0, scan_maxima[0], int(XYZscaling[3]))
    y_scan = np.linspace(0, scan_maxima[1], int(XYZscaling[6]))
    z_scan = np.linspace(0, scan_maxima[2], int(XYZscaling[9]))

    return(x_scan, y_scan, z_scan, voxel_size)

In [4]:
def resize_image_data(image_array, x_d, y_d, z_d):
    
    image_array_3D = image_array.reshape(z_d, y_d, x_d)
    return(image_array_3D)

In [5]:
def create_data_frame(image_array, x_scan, y_scan, z_scan):
    
    # create a mesh grid fo the heatmap
    X, Y, Z = np.meshgrid(x_scan, y_scan, z_scan)
    # Put all the data into a pandas data frame. 
    data_array = pd.DataFrame({'x':X.ravel(), 'y':Y.ravel(), 'z':Z.ravel(), 'Counts':image_array.ravel()})
    #the data_array needs to be pivoted to display properly in the heatmap.
    image_frame = data_array.pivot('z', 'y', 'x','Counts')
    return(image_frame)
    

In [7]:
file_name = ('I:\\Science\\Physics-Nanobiophotonics\\Group\\User Data\\' + 
             'ODMR\\ODMR and imaging\\20190902\\20190902_01.dat')

# Get the image data from the data file and the x, y and z parameters
image_array, meta_data, max_fluorescence = get_data(file_name)
# Generate the x, y, and z scans for the meshgrid. 
# Also generate the voxel size.
x_scan, y_scan, z_scan, voxel_size = xyz_scales(meta_data)
# Turn the image_array into a 3D array and then store this in a pandas data frame.
image_array_3D = resize_image_data(image_array, len(x_scan), 
                                  len(y_scan), len(z_scan)) 


(7686, 126)


In [7]:
import napari

In [8]:
# Generate a qt gui environment for napari. This is required to make napari run.
%gui qt

In [3]:
# This uses pandas array to generate a 3D image in napari.
# A graphics card is required for napari.
viewer = napari.view_image(image_array_3d)